In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../cleaned_hate_crime.csv")

In [3]:
df.head()

,region_name,offender_race,grouped_total_offender_count,grouped_victim_count,generalized_offense_name,generalized_location_name,generalized_bias_desc
0,Midwest,White,Few,Few,violent crimes,Residence,race
1,Northeast,White,Few,Few,violent crimes,Miscellaneous,race
2,Northeast,White,Few,Few,violent crimes,Miscellaneous,race
3,South,Black or African American,Several,Few,violent crimes,Public Place,race
4,Northeast,Black or African American,Few,Few,violent crimes,Miscellaneous,race


In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(multi_class="multinomial", solver='lbfgs', max_iter=1000)